In [ ]:
# imports libraries and dependencies
from selenium import webdriver
import time
import os
import pandas as pd
import random
from dotenv import load_dotenv

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [ ]:
# url used to get the internship postings which is already filtered
url2 = "https://www.indeed.com/jobs?q=software+engineer+intern&l=United+States"

In [46]:
# sets up the chrome driver and opens the webpage
load_dotenv()
service = Service(os.getenv("CHROMEDRIVER_PATH"))
driver = webdriver.Chrome(service=service)
driver.get(url2)
driver.implicitly_wait(10)

In [ ]:
# data to collect
companyNames = []
titles = []
jobLocations = []
links = []

In [ ]:
# extracts the data on each page and navigates all pages that has the postings
while True:
    try:
        jobElements = driver.find_elements(By.CLASS_NAME, "cardOutline")
        
        time.sleep(3)
        
        for job in jobElements:
            companyName = job.find_element(By.CSS_SELECTOR, '[data-testid="company-name"]').text
            print(companyName)
            companyNames.append(companyName)
            title = job.find_element(By.CLASS_NAME, "jobTitle").text
            titles.append(title)
            jobLocation = job.find_element(By.CSS_SELECTOR, '[data-testid="text-location"]').text
            jobLocations.append(jobLocation)
            link = job.find_element(By.CLASS_NAME, "jobTitle").find_element(By.TAG_NAME, 'a').get_attribute('href')
            links.append(link)
        
        nextPage = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//a[@aria-label='Next Page']"))
        )
        print(nextPage)
        nextPage.click()
        time.sleep(random.randint(3, 6))
    
    except (NoSuchElementException, TimeoutException):
        break
    
    except Exception as e:
        time.sleep(random.randint(2, 5))


In [ ]:
# convert data to pandas dataframe
companyNameData = pd.DataFrame(companyNames, columns=["Company Name"])
titleData = pd.DataFrame(titles, columns=["Title"])
jobLocationData = pd.DataFrame(jobLocations, columns=["Job Location"])
linkData = pd.DataFrame(links, columns=["Links"])

In [ ]:
# aggregates/concatenates all the data into one dataframe
jobData = pd.DataFrame()
jobData = pd.concat([companyNameData, titleData, jobLocationData, linkData], axis=1)
numColumns = (jobData.shape[1]*2)-1

for i in range(1, numColumns, 2):
    jobData.insert(i, f'Empty_{i}', '')
    
jobData.index = jobData.index+1

In [ ]:
jobData

In [ ]:
# transports data to a csv file
jobData.to_csv("Indeed_SWE_Intern_Jobs.csv")

In [ ]:
driver.quit()